# Dataset

In [9]:
# Load the dataset

# Network

In [12]:
class Denoising_Model(nn.Module):
    def __init__(self):
        super(denoising_model,self).__init__()
        self.encoder=nn.Sequential(
                      nn.Linear(inputSize, 770),  # There are 6040 users in movieLens-1M
                      nn.Tanh()
                      )

        self.decoder=nn.Sequential(
                      nn.Linear(770, inputSize),
                      nn.Tanh()
                      )


    def forward(self,x):
        x=self.encoder(x)
        x=self.decoder(x)
        return x

In [19]:
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

def train():
    if torch.cuda.is_available()==True:
        device="cuda:0"
    else:
        device ="cpu"

    model=Denoising_Model().to(device)
    criterion=nn.MSELoss()
    optimizer=optim.SGD(model.parameters(),lr=0.07,weight_decay=0.05)


    epochs=20
    l=len(train_loader)
    epoch_loss=0
    running_loss=0
    for epoch in range(epochs):
        for dirty,clean,label in tqdm((train_loader)):
            dirty=dirty.view(dirty.size(0),-1).type(torch.FloatTensor)
            clean=clean.view(clean.size(0),-1).type(torch.FloatTensor)
            dirty,clean=dirty.to(device),clean.to(device)
            # Forward Pass
            output=model(dirty)
            loss=criterion(output,clean)
            #Backward Pass---------------------
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss+=loss.item()
            epoch_loss+=loss.item()
        print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss))

In [20]:
from sklearn.metrics import mean_squared_error
from math import sqrt


def test():
    criterion=nn.MSECriterion()
    noRatings = 0
    input, target, minibatch = {}, {}, {}
    # TODO: Replace 'out'/'inp' 
    y_true = [test_dataset[i]['out'].item() for i in range(len(test_dataset))]
    y_pred = []

    trial_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)
    for bidx, batch in enumerate(test_loader):
            x_train = batch['inp']
            y_pred.append(net(x_train))

    y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

    rmse = sqrt(mean_squared_error(y_true, y_pred))
    print("RMSE: ", rmse)
